In [42]:
import pandas as pd
import numpy as np

In [43]:
pd.options.display.max_columns = None

In [44]:
df = pd.read_csv('dpe-69.csv')

/tmp/ipykernel_9913/2124654301.py:1: DtypeWarning: Columns (35,36,37,43,92,93,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dpe-69.csv')


In [45]:
df.shape

(332834, 131)

In [46]:
df.head()

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,date_etablissement_dpe,date_arrete_tarifs_energies,commentaires_ameliorations_recommandations,explication_personnalisee,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,tr002_type_batiment_id,secteur_activite,tr012_categorie_erp_id,tr013_type_erp_id,annee_construction,surface_habitable,surface_thermique_lot,tv016_departement_id,commune,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,code_postal,code_insee_commune,code_insee_commune_actualise,numero_lot,quote_part,nom_centre_commercial,surface_commerciale_contractuelle,portee_dpe_batiment,partie_batiment,shon,surface_utile,surface_thermique_parties_communes,en_souterrain,en_surface,nombre_niveaux,nombre_circulations_verticales,nombre_boutiques,presence_verriere,surface_verriere,type_vitrage_verriere,nombre_entrees_avec_sas,nombre_entrees_sans_sas,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_l4,geo_l5,tr001_modele_dpe_code,tr001_modele_dpe_type_id,tr001_modele_dpe_modele,tr001_modele_dpe_description,tr001_modele_dpe_fichier_vierge,tr001_modele_dpe_est_efface,tr001_modele_dpe_type,tr001_modele_dpe_type_libelle,tr001_modele_dpe_type_ordre,tr002_type_batiment_code,tr002_type_batiment_description,tr002_type_batiment_libelle,tr002_type_batiment_est_efface,tr002_type_batiment_ordre,tr002_type_batiment_simulateur,tr012_categorie_erp_code,tr012_categorie_erp_categorie,tr012_categorie_erp_groupe,tr012_categorie_erp_est_efface,tr013_type_erp_code,tr013_type_erp_type,tr013_type_erp_categorie_id,tr013_type_erp_est_efface,tr013_type_erp_categorie,tv016_departement_code,tv016_departement_departement,tv017_zone_hiver_id,tv018_zone_ete_id,tv016_departement_altmin,tv016_departement_altmax,tv016_departement_nref,tv016_departement_dhref,tv016_departement_pref,tv016_departement_c2,tv016_departement_c3,tv016_departement_c4,tv016_departement_t_ext_basse,tv016_departement_e,tv016_departement_fch,tv016_departement_fecs_ancienne_m_i,tv016_departement_fecs_recente_m_i,tv016_departement_fecs_solaire_m_i,tv016_departement_fecs_ancienne_i_c,tv016_departement_fecs_recente_i_c,tv017_zone_hiver_code,tv017_zone_hiver_t_ext_moyen,tv017_zone_hiver_peta_cw,tv017_zone_hiver_dh14,tv017_zone_hiver_prs1,tv018_zone_ete_code,tv018_zone_ete_sclim_inf_150,tv018_zone_ete_sclim_sup_150,tv018_zone_ete_rclim_autres_etages,tv018_zone_ete_rclim_dernier_etage
0,134,1369V1000001F,227,2,1,3CL - DPE,V2012,NaN,NaN,2013-04-10,2013-04-10,2011-08-15,Logement non occupé depuis plus d'une année. A...,NaN,416.56,F,21.73,D,1,Habitation (Maisons individuelles),NaN,NaN,1980,100.71,100.71,69,DÉCINES-CHARPIEU,50,rue,de Verdun,28,NaN,NaN,NaN,NaN,69150,69275,69275,sans objet,NaN,NaN,NaN,2.0,NaN,0.00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Socotec certification,"3, av du Centre 78280 Guyancourt",0.0,0,2013-04-10 04:00:00,3.480600,46.828209,0.73,poi.bus_stop,Avenue de Verdun / Rue des Pendants (Arrêt de ...,https://osm.org/node/1804339871,NaN,NaN,V1,1,6.1,Pour les bâtiments à usage principal d’h...,1_V1_DPE_6.1.pdf,0,Vente,Vente,1,TR002_001,Maison Individuelle,Maison,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,69 - Rhône,1,3,0,1200,4900,54000,80,340.0,1.5,NaN,-10,392.0,25.2,53.5,67.8,90.0,29.0,43.0,H1,6.58,10.5,42030,3.6,Ec,4,6,3.0,4
1,147,1369L1000001E,3825,3,9,3CL,"3CL-DPE, version 1.3",NaN,NaN,2013-04-09,2013-04-10,2011-09-15,NaN,NaN,254.00,E,47.00,E,1,Maison Individuelle,NaN,NaN,1978,97.00,97.00,69,COURS-LA-VILLE,NaN,NaN,13 

In [47]:
print(df.columns.tolist())

['id', 'numero_dpe', 'usr_diagnostiqueur_id', 'usr_logiciel_id', 'tr001_modele_dpe_id', 'nom_methode_dpe', 'version_methode_dpe', 'nom_methode_etude_thermique', 'version_methode_etude_thermique', 'date_visite_diagnostiqueur', 'date_etablissement_dpe', 'date_arrete_tarifs_energies', 'commentaires_ameliorations_recommandations', 'explication_personnalisee', 'consommation_energie', 'classe_consommation_energie', 'estimation_ges', 'classe_estimation_ges', 'tr002_type_batiment_id', 'secteur_activite', 'tr012_categorie_erp_id', 'tr013_type_erp_id', 'annee_construction', 'surface_habitable', 'surface_thermique_lot', 'tv016_departement_id', 'commune', 'arrondissement', 'type_voie', 'nom_rue', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'code_postal', 'code_insee_commune', 'code_insee_commune_actualise', 'numero_lot', 'quote_part', 'nom_centre_commercial', 'surface_commerciale_contractuelle', 'portee_dpe_batiment', 'partie_batiment', 'shon', 'surface_utile', 'surface_thermique_parti

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332834 entries, 0 to 332833
Columns: 131 entries, id to tv018_zone_ete_rclim_dernier_etage
dtypes: float64(49), int64(28), object(54)
memory usage: 332.7+ MB


In [49]:
df.describe(include='all')

,id,numero_dpe,usr_diagnostiqueur_id,usr_logiciel_id,tr001_modele_dpe_id,nom_methode_dpe,version_methode_dpe,nom_methode_etude_thermique,version_methode_etude_thermique,date_visite_diagnostiqueur,date_etablissement_dpe,date_arrete_tarifs_energies,commentaires_ameliorations_recommandations,explication_personnalisee,consommation_energie,classe_consommation_energie,estimation_ges,classe_estimation_ges,tr002_type_batiment_id,secteur_activite,tr012_categorie_erp_id,tr013_type_erp_id,annee_construction,surface_habitable,surface_thermique_lot,tv016_departement_id,commune,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,code_postal,code_insee_commune,code_insee_commune_actualise,numero_lot,quote_part,nom_centre_commercial,surface_commerciale_contractuelle,portee_dpe_batiment,partie_batiment,shon,surface_utile,surface_thermique_parties_communes,en_souterrain,en_surface,nombre_niveaux,nombre_circulations_verticales,nombre_boutiques,presence_verriere,surface_verriere,type_vitrage_verriere,nombre_entrees_avec_sas,nombre_entrees_sans_sas,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,surface_planchers_hauts_deperditifs,surface_planchers_bas_deperditifs,surface_parois_verticales_opaques_deperditives,etat_avancement,organisme_certificateur,adresse_organisme_certificateur,dpe_vierge,est_efface,date_reception_dpe,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_l4,geo_l5,tr001_modele_dpe_code,tr001_modele_dpe_type_id,tr001_modele_dpe_modele,tr001_modele_dpe_description,tr001_modele_dpe_fichier_vierge,tr001_modele_dpe_est_efface,tr001_modele_dpe_type,tr001_modele_dpe_type_libelle,tr001_modele_dpe_type_ordre,tr002_type_batiment_code,tr002_type_batiment_description,tr002_type_batiment_libelle,tr002_type_batiment_est_efface,tr002_type_batiment_ordre,tr002_type_batiment_simulateur,tr012_categorie_erp_code,tr012_categorie_erp_categorie,tr012_categorie_erp_groupe,tr012_categorie_erp_est_efface,tr013_type_erp_code,tr013_type_erp_type,tr013_type_erp_categorie_id,tr013_type_erp_est_efface,tr013_type_erp_categorie,tv016_departement_code,tv016_departement_departement,tv017_zone_hiver_id,tv018_zone_ete_id,tv016_departement_altmin,tv016_departement_altmax,tv016_departement_nref,tv016_departement_dhref,tv016_departement_pref,tv016_departement_c2,tv016_departement_c3,tv016_departement_c4,tv016_departement_t_ext_basse,tv016_departement_e,tv016_departement_fch,tv016_departement_fecs_ancienne_m_i,tv016_departement_fecs_recente_m_i,tv016_departement_fecs_solaire_m_i,tv016_departement_fecs_ancienne_i_c,tv016_departement_fecs_recente_i_c,tv017_zone_hiver_code,tv017_zone_hiver_t_ext_moyen,tv017_zone_hiver_peta_cw,tv017_zone_hiver_dh14,tv017_zone_hiver_prs1,tv018_zone_ete_code,tv018_zone_ete_sclim_inf_150,tv018_zone_ete_sclim_sup_150,tv018_zone_ete_rclim_autres_etages,tv018_zone_ete_rclim_dernier_etage
count,3.328340e+05,332834,332834.000000,332834.000000,332834.000000,332834,251448,55934,52632,332834,332834,332834,53090,29501,3.328340e+05,332834,3.328340e+05,332834,332834.000000,26224,2047.000000,16443.000000,332834.000000,332828.000000,332828.000000,332834.0,332614,117626,72243,332754,92482,26055,100146,47120,8884,332768,332768.0,332768.0,159813,11895,0.0,45027.000000,267208.000000,45,250526.000000,267208.000000,267208.000000,250526.0,250526.000000,267208.000000,250526.0,250526.0,267208.000000,250526.0,0.0,250526.0,250531.0,257683.000000,257683.000000,257683.000000,257683.000000,257683.000000,257683.000000,332834.0,331746,319712,307844.000000,332834.0,332834,310283.000000,310283.000000,332834.000000,310283,310283,306909,300444,829,332834,332834.000000,332834,332834,332834,332834.0,332834,332834,332834.000000,332834,332834,332834,332834.0,332834.000000,332834.000000,2047,2047,2047,2047.0,16443,16443,16443.000000,16443.0,16443,332834.0,332834,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,332834.0,0.0,332834.0,332834.0,3.328340e+05,332834.0,3.328340e+05,332834.0,

In [50]:
mask = df['code_insee_commune']==69123
df = df[mask]
df.shape

(60530, 131)

In [51]:
col_to_keep = ['classe_consommation_energie', 'secteur_activite', 'annee_construction', 'surface_habitable', 'arrondissement', 'type_voie', 'nom_rue', 'numero_rue', 'batiment', 'escalier', 'etage', 'porte', 'nombre_niveaux', 'surface_baies_orientees_nord', 'surface_baies_orientees_est_ouest', 'surface_baies_orientees_sud', 'longitude', 'latitude', 'tr001_modele_dpe_type_libelle']

In [52]:
df = df[col_to_keep]
df.shape

(60530, 19)

In [53]:
df['classe_consommation_energie'].unique()

array(['D', 'E', 'F', 'A', 'G', 'C', 'N', 'B'], dtype=object)

In [54]:
mask = df['classe_consommation_energie']=='N'
df = df[~mask]
df.shape

(46578, 19)

In [55]:
df['classe_consommation_energie'].unique()

array(['D', 'E', 'F', 'A', 'G', 'C', 'B'], dtype=object)

In [56]:
df['secteur_activite'].unique()

array(['Habitation (parties privatives d immeuble col', nan,
       "Habitation (Parties privatives d'immeuble col",
       'Habitation (maisons individuelles)', 'Bureaux',
       'Habitation (Maisons individuelles)', 'Local commercial', 'Garage',
       'Residence', 'Habitation (Parties communes)', 'Commerce',
       'Habitation (parties communes)', 'Autres',
       "Immeuble d'habitation collectif",
       'Batiment(s) a usage de commerces et d habitat',
       'Batiment a usage de bureaux et d habitation',
       'Etablissements sociaux',
       'Habitation (maison en copropriete horizontale',
       'Chambres privatives', 'Appartement', 'Etablissement sociaux',
       'Habitation (maison individuelle)'], dtype=object)

In [57]:
activite_to_dump = ['Bureaux',
       'Local commercial', 'Garage',
       'Commerce',
       'Autres',
       'Etablissements sociaux',
       'Chambres privatives', 'Etablissement sociaux']

In [58]:
df['mask_activite'] = df['secteur_activite'].map(lambda x : 0 if x in activite_to_dump else 1)
mask = df['mask_activite']==1
df = df[mask]
df.shape

(46561, 20)

In [59]:
df.head()

,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite
4096,D,Habitation (parties privatives d immeuble col,1974,63.00,09,RUE,DU CHATEAU DE LA DUCHERE,121,NaN,NaN,1er Etage,NaN,1.0,NaN,NaN,NaN,4.792479,45.780395,Location,1
4111,D,NaN,1999,91.27,Sans objet,non communiquée,32 Rue Saint Romain,NaN,NaN,NaN,NaN,NaN,0.0,0.00,4.95,0.60,4.864825,45.741007,Copropriété,1
4122,E,Habitation (parties privatives d immeuble col,1974,32.00,02,QUAI,DE TILSIT,25,NaN,NaN,Rez de chaussee,NaN,1.0,1.70,2.16,0.00,4.826129,45.755381,Location,1
4123,D,Habitation (Parties privatives d'immeuble col,1990,84.87,03,NaN,277 RUE DUGESCLIN,NaN,APPT,NaN,3ème,NaN,0.0,0.00,0.00,0.00,4.849889,45.755710,Location,1
4128,D,NaN,1948,78.72,0,NaN,33 Avenue Jean François Raclet,NC,NC,NaN,RDC,NaN,1.0,5.59,0.00,5.59,4.840204,45.734100,Copropriété,1


In [60]:
df.describe(include='all')

,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite
count,46561,4998,46561.000000,46561.000000,13022,5067,46558,11559,7595,25817,12564,2080,46554.000000,44801.000000,44801.000000,44801.000000,46136.000000,46136.000000,46561,46561.0
unique,7,13,NaN,NaN,17,100,27611,805,417,5573,302,614,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN
top,D,Habitation (parties privatives d immeuble col,NaN,NaN,0,rue,60 MONTEE DE L'OBSERVANCE,Non communiqué,NC,Etage 1,NC,Gauche,NaN,NaN,NaN,NaN,NaN,NaN,Vente,NaN
freq,16504,2834,NaN,NaN,6463,1586,503,4226,3889,1824,1644,198,NaN,NaN,NaN,NaN,NaN,NaN,25097,NaN
mean,NaN,NaN,1842.722536,73.456717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.148902,2.133508,3.848943,2.382685,4.842961,45.756682,NaN,1.0
std,NaN,NaN,479.535910,187.286699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.121492,28.249898,12.741384,17.711476,0.045973,0.050878,NaN,0.0
min,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.368634,43.663685,NaN,1.0
25%,NaN,NaN,1947.000000,42.010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,4.828467,45.747165,NaN,1.0
50%,NaN,NaN,1948.000000,60.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,4.844629,45.756031,NaN,1.0
75%,NaN,NaN,1989.000000,79.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.340000,5.330000,1.280000,4.862494,45.768609,NaN,1.0


In [61]:
df.head()

,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite
4096,D,Habitation (parties privatives d immeuble col,1974,63.00,09,RUE,DU CHATEAU DE LA DUCHERE,121,NaN,NaN,1er Etage,NaN,1.0,NaN,NaN,NaN,4.792479,45.780395,Location,1
4111,D,NaN,1999,91.27,Sans objet,non communiquée,32 Rue Saint Romain,NaN,NaN,NaN,NaN,NaN,0.0,0.00,4.95,0.60,4.864825,45.741007,Copropriété,1
4122,E,Habitation (parties privatives d immeuble col,1974,32.00,02,QUAI,DE TILSIT,25,NaN,NaN,Rez de chaussee,NaN,1.0,1.70,2.16,0.00,4.826129,45.755381,Location,1
4123,D,Habitation (Parties privatives d'immeuble col,1990,84.87,03,NaN,277 RUE DUGESCLIN,NaN,APPT,NaN,3ème,NaN,0.0,0.00,0.00,0.00,4.849889,45.755710,Location,1
4128,D,NaN,1948,78.72,0,NaN,33 Avenue Jean François Raclet,NC,NC,NaN,RDC,NaN,1.0,5.59,0.00,5.59,4.840204,45.734100,Copropriété,1


In [62]:
#determine orientation
def orientation(n, eo, s):
    try :
        maxi = max(n, eo, s)
    except :
        return None

    if maxi == 0 :
        return None
    if maxi == n :
        return 'N'
    if maxi == eo :
        return 'EO'
    if maxi == s :
        return 'S'
    


In [63]:
df['orientation'] = list(map(orientation, df['surface_baies_orientees_nord'], df['surface_baies_orientees_est_ouest'], df['surface_baies_orientees_sud']))

In [64]:
df.head()

,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite,orientation
4096,D,Habitation (parties privatives d immeuble col,1974,63.00,09,RUE,DU CHATEAU DE LA DUCHERE,121,NaN,NaN,1er Etage,NaN,1.0,NaN,NaN,NaN,4.792479,45.780395,Location,1,None
4111,D,NaN,1999,91.27,Sans objet,non communiquée,32 Rue Saint Romain,NaN,NaN,NaN,NaN,NaN,0.0,0.00,4.95,0.60,4.864825,45.741007,Copropriété,1,EO
4122,E,Habitation (parties privatives d immeuble col,1974,32.00,02,QUAI,DE TILSIT,25,NaN,NaN,Rez de chaussee,NaN,1.0,1.70,2.16,0.00,4.826129,45.755381,Location,1,EO
4123,D,Habitation (Parties privatives d'immeuble col,1990,84.87,03,NaN,277 RUE DUGESCLIN,NaN,APPT,NaN,3ème,NaN,0.0,0.00,0.00,0.00,4.849889,45.755710,Location,1,None
4128,D,NaN,1948,78.72,0,NaN,33 Avenue Jean François Raclet,NC,NC,NaN,RDC,NaN,1.0,5.59,0.00,5.59,4.840204,45.734100,Copropriété,1,N


In [65]:
df['orientation'].value_counts()

EO    14462
S      7556
N      6215
Name: orientation, dtype: int64

In [66]:
df['orientation'].isna().sum()

18328

In [67]:
df.describe(include='all')

,classe_consommation_energie,secteur_activite,annee_construction,surface_habitable,arrondissement,type_voie,nom_rue,numero_rue,batiment,escalier,etage,porte,nombre_niveaux,surface_baies_orientees_nord,surface_baies_orientees_est_ouest,surface_baies_orientees_sud,longitude,latitude,tr001_modele_dpe_type_libelle,mask_activite,orientation
count,46561,4998,46561.000000,46561.000000,13022,5067,46558,11559,7595,25817,12564,2080,46554.000000,44801.000000,44801.000000,44801.000000,46136.000000,46136.000000,46561,46561.0,28233
unique,7,13,NaN,NaN,17,100,27611,805,417,5573,302,614,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,3
top,D,Habitation (parties privatives d immeuble col,NaN,NaN,0,rue,60 MONTEE DE L'OBSERVANCE,Non communiqué,NC,Etage 1,NC,Gauche,NaN,NaN,NaN,NaN,NaN,NaN,Vente,NaN,EO
freq,16504,2834,NaN,NaN,6463,1586,503,4226,3889,1824,1644,198,NaN,NaN,NaN,NaN,NaN,NaN,25097,NaN,14462
mean,NaN,NaN,1842.722536,73.456717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.148902,2.133508,3.848943,2.382685,4.842961,45.756682,NaN,1.0,NaN
std,NaN,NaN,479.535910,187.286699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.121492,28.249898,12.741384,17.711476,0.045973,0.050878,NaN,0.0,NaN
min,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.368634,43.663685,NaN,1.0,NaN
25%,NaN,NaN,1947.000000,42.010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,4.828467,45.747165,NaN,1.0,NaN
50%,NaN,NaN,1948.000000,60.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,4.844629,45.756031,NaN,1.0,NaN
75%,NaN,NaN,1989.000000,79.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.340000,5.330000,1.280000,4.862494,45.768609,NaN,1.0,NaN
